# Comparing Temperatures and Luminosities Between Analyses

This notebook measures global temperatures and luminosities for the samples we are using to verify our method, then compares them to the measurements from literature. While the focus of this work is the measurement of galaxy cluster masses, temperatures and luminosities play an important role in the construction of X-ray mass-observable scaling relations - thus they must also be shown to be consistent with previous work. 

## Import Statements

In [1]:
import pandas as pd
import numpy as np
from astropy.units import Quantity, UnitConversionError
import matplotlib.pyplot as plt
from typing import Union, List
from shutil import rmtree
import os

# This adds the directory above to the path, allowing me to import the common functions that I've written in
#  common.py - this just saves me repeating boring code and makes sure its all consistent
import sys
sys.path.insert(0, '..')
from common import xcs3p_colour, xcs_cosmo, direct_comparison_plot, find_lims, fit_comp

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 80
# This is a bodge that will only work because xga_output in notebooks has already been defined, XGA
#  will be made to handle this more gracefully at some point
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/')[0]+'notebooks/xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.samples import ClusterSample
from xga.sources import GalaxyCluster
from xga.sources import BaseSource
from xga.xspec import single_temp_apec

# This is a bit cheeky, but suppresses the warnings that XGA spits out (they are 
#  useful, but not when I'm trying to present this notebook on GitHub)
import warnings
warnings.filterwarnings('ignore')

## Reading in Sample Files and Declaring XGA ClusterSamples

This subsection involves reading in the sample files of the four test samples (described in [the sample properties notebook](sample_properties.ipynb)), then setting up separate XGA ClusterSample instances (see [the documentation](https://xga.readthedocs.io/en/latest/notebooks/tutorials/sources_samples.html) for an introduction to XGA source and sample objects.

We impose an additional cleaning step on each sample, where we make sure that (for each XMM observation initially associated with a source) at least 70% of a cluster's $R_{500}$ falls on that observation - if this requirement is not met then the observation is excluded. These requirements are set with the `clean_obs=True`, `clean_obs_reg='r500'`, and `clean_obs_threshold=0.7` arguments when a ClusterSample instance is declared.

### SDSSRM-XCS Volume Limited

This is the recent SDSSRM-XCS sample. The temperatures and luminosities are measured by the XCS luminosity-temperature pipeline, and with this we demonstrate that XGA temperatures and luminosities are consistent with existing XCS results.

In order to achieve maximum consistency, we use the XAPA coordinates as the central position for spectrum generation (turning off the XGA peak finder with `use_peak=False`). We have also made sure to use the same cosmology.

Two files are read in here, one with the results of the XCS3P luminosity-temperature pipeline measured within $R_{500}$ and another with results measured within $R_{2500}$. Not all clusters in the $R_{500}$ sample have a corresponding $R_{2500}$.

In [2]:
xcs3p = pd.read_csv("../../sample_files/xcs3p_sdssrm_vol_lim_temperr_25%_clusters.csv")
xcs3p_r2500 = pd.read_csv("../../sample_files/xcs3p_sdssrm_r2500_props_nonvol_lim_temperr_100%_clusters.csv")

In [3]:
# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = xcs3p['xapa_ra'].values
dec = xcs3p['xapa_dec'].values
z = xcs3p['z'].values
# Not using the IAU names in XCS_NAME column, its easier for me to use the name based on redMaPPer ID
n = xcs3p['name'].values
# In arcminutes, ClusterSample declaration will convert to kpc using the provided cosmology
r500 = Quantity(xcs3p['r500'].values, 'arcmin')
# Not likely to use richness in this notebook, but I'm putting it in the sample object anyway
r = xcs3p['richness'].values
r_err = xcs3p['richness_err'].values

# Declaring the actual ClusterSample instance for the XCS sample
xcs_srcs = ClusterSample(ra, dec, z, n, r500=r500, richness=r, richness_err=r_err, cosmology=xcs_cosmo, 
                         load_fits=True, use_peak=False, clean_obs=True, clean_obs_reg='r500', 
                         clean_obs_threshold=0.7)

Setting up Galaxy Clusters: 100%|███████████████████████████████████████████| 150/150 [06:50<00:00,  2.74s/it]


Also define another sample with a subset of those clusters, specifically for those clusters that also have an $R_{2500}$ value as measured by the XCS3P luminosity temperature pipeline. A separate sample in this case makes it easier to run XGA measurements of temperature and luminosity within $R_{2500}$.

In [4]:
# Read in the R2500 values from a different file, as they result from a different analysis
r2500 = []
with_r2500 = []
for row_ind, row in xcs3p.iterrows():
    rel_row = xcs3p_r2500[xcs3p_r2500['MEM_MATCH_ID'] == row['MEM_MATCH_ID']]
    # Not all the R500 clusters have a valid R2500, so accounting for that
    try:
        r2500.append(rel_row['r2500'].values[0])
        with_r2500.append(True)
    except IndexError:
        with_r2500.append(False)
r2500 = Quantity(r2500, 'arcmin')
with_r2500 = np.array(with_r2500)

# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = xcs3p['xapa_ra'].values[with_r2500]
dec = xcs3p['xapa_dec'].values[with_r2500]
z = xcs3p['z'].values[with_r2500]
# Not using the IAU names in XCS_NAME column, its easier for me to use the name based on redMaPPer ID
n = xcs3p['name'].values[with_r2500]
# Not likely to use richness in this notebook, but I'm putting it in the sample object anyway
r = xcs3p['richness'].values[with_r2500]
r_err = xcs3p['richness_err'].values[with_r2500]
# In arcminutes, ClusterSample declaration will convert to kpc using the provided cosmology
r500 = Quantity(xcs3p['r500'].values, 'arcmin')[with_r2500]

# Slightly annoying but at the moment I have only let XGA take one set of factors for deciding a background 
#  region, so I also have to define a blanket one for this. As this will be used to measure R2500 quantities 
xcs_srcs_subset = ClusterSample(ra, dec, z, n, r500=r500, r2500=r2500, richness=r, richness_err=r_err, 
                                cosmology=xcs_cosmo, load_fits=True, use_peak=False, clean_obs=True, 
                                clean_obs_reg='r500', clean_obs_threshold=0.7, back_inn_rad_factor=2, 
                                back_out_rad_factor=3)

Setting up Galaxy Clusters: 100%|███████████████████████████████████████████| 133/133 [03:53<00:00,  1.76s/it]


## Running $T_{\rm{X}}$ and $L_{\rm{X}}$ Measurements

The XGA XSPEC functions that we use here all automatically call the XGA SAS interface, so the necessary spectra are generated before the fits begin. As the different samples measure properties within different spatial regions, all the function calls differ slightly.

The results of the fits are stored within the indivual source objects that make up each sample.

###  Measurements within $R_{500}$

For our comparisons here we wish to measure the temperature ($T_{\rm{X}}$) and luminosity ($L_\rm{X}$; both in the 0.5-2.0 keV and bolometric keV energy bands) within $R_{\rm{500}}$. We fit a `constant*tbabs*apec` model; with the choices for absorption (`tbabs`) and plasma emission (`tbabs`) consistent with the XCS analysis, though the addition of a multiplicative constant to manage differences in sensitivity is different from the original analysis.

In [5]:
single_temp_apec(xcs_srcs, xcs_srcs.r500, timeout=Quantity(4, 'hr'))

Running XSPEC Fits: 100%|█████████████████████████████████████████████████| 150/150 [1:39:34<00:00, 39.83s/it]


### Measurements within 0.15-1$R_{500}$

Though we aren't comparing to any core-excised measurements for the SDSSRM-XCS sample, we might as well generate and fit spectra with the core excised. That way we can also create scaling relations with core-excised properties if we want to later.

In [7]:
single_temp_apec(xcs_srcs, xcs_srcs.r500, inner_radius=0.15*xcs_srcs.r500, timeout=Quantity(4, 'hr'))

Running XSPEC Fits: 100%|█████████████████████████████████████████████████| 150/150 [2:06:29<00:00, 50.60s/it]


### Measurements within $R_{2500}$

In [8]:
single_temp_apec(xcs_srcs_subset, xcs_srcs_subset.r2500, timeout=Quantity(4, 'hr'))

Running XSPEC Fits: 100%|███████████████████████████████████████████████████| 133/133 [41:23<00:00, 18.67s/it]


## Retrieving $T_{\rm{X}}$ and $L_{\rm{X}}$ measurements from the samples

We must extract and judge the quality of the temperature and luminosity measurements that we have made for each of the samples, then later on we will be able to directly compare them. At the same time we make sure that the results from literature are formatted in such a way that we can easily compare them.

### Within $R_{500}$

In [9]:
sdss_tx_all = xcs_srcs.Tx(xcs_srcs.r500, quality_checks=False).value
sdss_tx = xcs_srcs.Tx(xcs_srcs.r500, quality_checks=True)

sdss_lxbol_all = xcs_srcs.Lx(xcs_srcs.r500, quality_checks=False, lo_en=Quantity(0.01, 'keV'), 
                             hi_en=Quantity(100.0, 'keV'))
sdss_lxbol = xcs_srcs.Lx(xcs_srcs.r500, quality_checks=True, lo_en=Quantity(0.01, 'keV'), 
                         hi_en=Quantity(100.0, 'keV'))

sdss_lx_all = xcs_srcs.Lx(xcs_srcs.r500, quality_checks=False, lo_en=Quantity(0.5, 'keV'), 
                             hi_en=Quantity(2.0, 'keV'))
sdss_lx = xcs_srcs.Lx(xcs_srcs.r500, quality_checks=True, lo_en=Quantity(0.5, 'keV'), 
                         hi_en=Quantity(2.0, 'keV'))

In [10]:
xcs3p_tx = []
xcs3p_lx52 = []
xcs3p_lxbol = []

for src in xcs_srcs:
    n = src.name
    rel_row = xcs3p[xcs3p['name'] == n].iloc[0]
    xcs3p_tx.append(Quantity(rel_row[['Tx', 'Tx-', 'Tx+']].values, 'keV'))
    xcs3p_lx52.append(Quantity(rel_row[['Lx52', 'Lx52-', 'Lx52+']].values*1e+44, 'erg/s'))
    xcs3p_lxbol.append(Quantity(rel_row[['Lx', 'Lx-', 'Lx+']].values*1e+44, 'erg/s'))  

xcs3p_tx = Quantity(xcs3p_tx)
xcs3p_lx52 = Quantity(xcs3p_lx52)
xcs3p_lxbol = Quantity(xcs3p_lxbol)

### Within $R_{2500}$

In [11]:
sdss_tx2500_all = xcs_srcs_subset.Tx(xcs_srcs_subset.r2500, quality_checks=False).value
sdss_tx2500 = xcs_srcs_subset.Tx(xcs_srcs_subset.r2500, quality_checks=True)

sdss_lxbol2500_all = xcs_srcs_subset.Lx(xcs_srcs_subset.r2500, quality_checks=False, lo_en=Quantity(0.01, 'keV'), 
                             hi_en=Quantity(100.0, 'keV'))
sdss_lxbol2500 = xcs_srcs_subset.Lx(xcs_srcs_subset.r2500, quality_checks=True, lo_en=Quantity(0.01, 'keV'), 
                         hi_en=Quantity(100.0, 'keV'))

sdss_lx2500_all = xcs_srcs_subset.Lx(xcs_srcs_subset.r2500, quality_checks=False, lo_en=Quantity(0.5, 'keV'), 
                             hi_en=Quantity(2.0, 'keV'))
sdss_lx2500 = xcs_srcs_subset.Lx(xcs_srcs_subset.r2500, quality_checks=True, lo_en=Quantity(0.5, 'keV'), 
                         hi_en=Quantity(2.0, 'keV'))

In [12]:
xcs3p_tx2500 = []
xcs3p_lx522500 = []
xcs3p_lxbol2500 = []

for src in xcs_srcs_subset: 
    n = src.name
    rel_row = xcs3p_r2500[xcs3p_r2500['name'] == n].iloc[0]
    xcs3p_tx2500.append(Quantity(rel_row[['Tx', 'Tx-', 'Tx+']].values, 'keV'))
    xcs3p_lx522500.append(Quantity(rel_row[['Lx52', 'Lx52-', 'Lx52+']].values*1e+44, 'erg/s'))
    xcs3p_lxbol2500.append(Quantity(rel_row[['Lx', 'Lx-', 'Lx+']].values*1e+44, 'erg/s'))  

xcs3p_tx2500 = Quantity(xcs3p_tx2500)
xcs3p_lx522500 = Quantity(xcs3p_lx522500)
xcs3p_lxbol2500 = Quantity(xcs3p_lxbol2500)

## Direct comparisons between original and XGA measurements

This is the the point of this notebook, making direct comparisons of like for like (or as near as we can achieve) measurements from literature to measurements made using XGA. In each case we plot simple one-to-one comparisons, with a one-to-one dashed line in red to give a reference.

### Within $R_{500}$

We directly compare temperatures and luminosities (soft band and bolimetric) between the XCS3P pipeline and XGA.

In [13]:
print('{s} out of {t} XGA Tx measurements were successful'.format(s=np.isfinite(sdss_tx[:, 0]).sum(), 
                                                                  t=len(sdss_tx)))
print('{s} out of {t} XGA Lx52 measurements were successful'.format(s=np.isfinite(sdss_lx[:, 0]).sum(), 
                                                                    t=len(sdss_lx)))
print('{s} out of {t} XGA Lxbol measurements were successful'.format(s=np.isfinite(sdss_lxbol[:, 0]).sum(), 
                                                                    t=len(sdss_lxbol)))

149 out of 150 XGA Tx measurements were successful
149 out of 150 XGA Lx52 measurements were successful
149 out of 150 XGA Lxbol measurements were successful


In [17]:
xcs_srcs.names[np.isnan(sdss_tx[:, 0])]

array(['SDSSXCS-6307'], dtype='<U13')

#### $T_{\rm{X}, 500}$, $L^{\rm{0.5-2.0}}_{\rm{X}, 500}$, and $L^{\rm{bol}}_{\rm{X}, 500}$ 

In [ ]:
direct_comparison_plot([xcs3p_tx, xcs3p_lx52, xcs3p_lxbol], [sdss_tx, sdss_lx, sdss_lxbol], 
                       [r"XCS3P $T_{\rm{X, 500}}$ [keV]", r"XCS3P $L^{\rm{0.5-2.0}}_{\rm{X, 500}}$ [erg s$^{-1}$]", 
                        r"XCS3P $L^{\rm{bol}}_{\rm{X, 500}}$ [erg s$^{-1}$]"], 
                       [r"XGA $T_{\rm{X, 500}}$ [keV]", r"XGA $L^{\rm{0.5-2.0}}_{\rm{X, 500}}$ [erg s$^{-1}$]", 
                        r"XGA $L^{\rm{bol}}_{\rm{X, 500}}$ [erg s$^{-1}$]"],
                       r"SDSSRM-XCS", (18, 6), ['linear', 'log', 'log'], ['linear', 'log', 'log'], 
                       savepath='../../outputs/figures/sdssxcs_Tx_Lxb_Lx52_comp.pdf')

In [ ]:
fit_comp(xcs3p_tx, sdss_tx, [[-5, 5], [-5, 5]], r"XCS3P $T_{\rm{X, 500}}$ [keV]", 
         r"XGA $T_{\rm{X, 500}}$ [keV]", Quantity(5.5, 'keV'),  num_steps=20000, view_chains=True, 
         linear=False, view_corner=True)

In [ ]:
fit_comp(xcs3p_lx52, sdss_lx, [[-10, 10], [-20, 20]], r"XCS3P $L^{\rm{0.5-2.0}}_{\rm{X, 500}}$ [erg s$^{-1}$]", 
         r"XGA $L^{\rm{0.5-2.0}}_{\rm{X, 500}}$ [erg s$^{-1}$]", Quantity(1e+44, 'erg/s'),  num_steps=20000, 
         view_chains=True, linear=False, view_corner=True, xscale='log', yscale='log')

In [ ]:
fit_comp(xcs3p_lxbol, sdss_lxbol, [[-10, 10], [-20, 20]], r"XCS3P $L^{\rm{bol}}_{\rm{X, 500}}$ [erg s$^{-1}$]", 
         r"XGA $L^{\rm{bol}}_{\rm{X, 500}}$ [erg s$^{-1}$]", Quantity(5e+44, 'erg/s'),  num_steps=20000, 
         view_chains=True, linear=True, view_corner=True, xscale='linear', yscale='linear')

### Within $R_{2500}$

In [ ]:
print('{s} out of {t} XGA Tx measurements were successful'.format(s=np.isfinite(sdss_tx2500[:, 0]).sum(), 
                                                                  t=len(sdss_tx2500)))
print('{s} out of {t} XGA Lx52 measurements were successful'.format(s=np.isfinite(sdss_lx2500[:, 0]).sum(), 
                                                                    t=len(sdss_lx2500)))
print('{s} out of {t} XGA Lxbol measurements were successful'.format(s=np.isfinite(sdss_lxbol2500[:, 0]).sum(), 
                                                                    t=len(sdss_lxbol2500)))

In [ ]:
direct_comparison_plot([xcs3p_tx2500, xcs3p_lx522500, xcs3p_lxbol2500], [sdss_tx2500, sdss_lx2500, sdss_lxbol2500], 
                       [r"XCS3P $T_{\rm{X, 2500}}$ [keV]", r"XCS3P $L^{\rm{0.5-2.0}}_{\rm{X, 2500}}$ [erg s$^{-1}$]", 
                        r"XCS3P $L^{\rm{bol}}_{\rm{X, 2500}}$ [erg s$^{-1}$]"], 
                       [r"XGA $T_{\rm{X, 2500}}$ [keV]", r"XGA $L^{\rm{0.5-2.0}}_{\rm{X, 2500}}$ [erg s$^{-1}$]", 
                        r"XGA $L^{\rm{bol}}_{\rm{X, 2500}}$ [erg s$^{-1}$]"],
                       r"SDSSRM-XCS", (18, 6), ['linear', 'log', 'log'], ['linear', 'log', 'log'], 
                       savepath='../../outputs/figures/sdssxcs_Tx_Lxb_Lx52_r2500_comp.pdf')

In [ ]:
fit_comp(xcs3p_tx2500, sdss_tx2500, [[-5, 5], [-5, 5]], r"XCS3P $T_{\rm{X, 2500}}$ [keV]", 
         r"XGA $T_{\rm{X, 2500}}$ [keV]", Quantity(5.5, 'keV'),  num_steps=20000, view_chains=True, 
         linear=True, view_corner=True)

In [ ]:
fit_comp(xcs3p_lx522500, sdss_lx2500, [[-10, 10], [-20, 20]], 
         r"XCS3P $L^{\rm{0.5-2.0}}_{\rm{X, 2500}}$ [erg s$^{-1}$]", 
         r"XGA $L^{\rm{0.5-2.0}}_{\rm{X, 2500}}$ [erg s$^{-1}$]", Quantity(1e+44, 'erg/s'),  num_steps=20000, 
         view_chains=True, linear=False, view_corner=True, xscale='log', yscale='log')